In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# src: https://github.com/SnehaDeshmukh28/IndianBailJudgments-1200/blob/main/indian_bail_judgments.csv 
df_raw = pd.read_csv("./dataset/indian_bail_judgements_raw.csv")

In [3]:
df_raw.head()

,case_id,case_title,court,date,judge,ipc_sections,bail_type,bail_cancellation_case,landmark_case,accused_name,...,facts,legal_issues,judgment_reason,summary,bias_flag,parity_argument_used,legal_principles_discussed,region,source_filename,special_laws
0,1,Jibangshu Paul vs National Investigation Agency,Gauhati High Court,2011-07-27,"Justice I. A. Ansari, Justice A. K. Goswami","['120B', '121', '121A']",Regular,True,True,Jibangshu Paul,...,Jibangshu Paul was apprehended carrying Rs. 32...,"['Whether fresh bail is needed when new, more ...",The court held that newly added serious UA(P) ...,Bail earlier granted to Jibangshu Paul under I...,False,False,['Addition of new penal sections requires fres...,Assam,case0001.PDF,NaN
1,2,State By The Superintendent Of Police vs Mehbo...,Madras High Court,1997-09-30,Justice E. Padmanabhan,"['376', '354', '343', '109', '220', '348', '33...",Regular,True,False,Mehboob Batcha and others (A1 to A11),...,The case involves custodial rape of a woman by...,['Whether suspension of sentence requires deta...,The court held that due to the grave nature of...,Madras High Court cancelled the bail granted t...,False,False,[],Tamil Nadu,case0002.PDF,NaN
2,3,Hyderali vs State Of Kerala,Kerala High Court,2008-08-05,Justice K. Hema,"['465', '468', '471', '474', '420', '511', '34']",Anticipatory,False,True,Hyderali,...,"Hyderali, a government contractor, was accused...",['Whether anticipatory bail can be granted whe...,The court ruled that despite other co-accused ...,Kerala High Court rejected anticipatory bail f...,False,True,['Custodial interrogation is legitimate and ma...,Kerala,case0003.PDF,NaN
3,4,Rashbehari Karmakar vs Indrajit Mukherjee And ...,Calcutta High Court,2002-07-12,"Justice P.K. Biswas, Justice S. Barman Roy","['326', '307', '120B', '201']",Regular,True,False,Indrajit Mukherjee,...,"The petitioner, a government employee, alleged...",['Whether bail should be cancelled under Secti...,The court found no cogent or overwhelming reas...,Calcutta High Court refused to cancel bail of ...,False,False,['Difference between grant and cancellation of...,West Bengal,case0004.PDF,NaN
4,5,Smt. Shankri Devi Age 62 Years vs Union Territ...,High Court of Jammu & Kashmir and Ladakh at Jammu,2023-06-12,Justice Mohan Lal,"['302', '34']",Regular,False,False,Smt. Shankri Devi,...,Shankri Devi and co-accused were charged with ...,['Whether the court’s earlier bail rejection o...,The court held that its earlier rejection of b...,Jammu & Kashmir High Court rejected the review...,False,False,['Bail once rejected cannot be reviewed except...,Jammu & Kashmir,case0005.PDF,NaN


Reqd: region, ipc section, bail_type, bail_cancellation_case, landmark_case, crime_type, facts, 
dont include: case_id, case_title, date, source_filename, special_laws, judges, court
let's see: judge

In [4]:
df_raw.describe()
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   case_id                      1200 non-null   int64 
 1   case_title                   1200 non-null   object
 2   court                        1200 non-null   object
 3   date                         1200 non-null   object
 4   judge                        1200 non-null   object
 5   ipc_sections                 1200 non-null   object
 6   bail_type                    1200 non-null   object
 7   bail_cancellation_case       1200 non-null   bool  
 8   landmark_case                1200 non-null   bool  
 9   accused_name                 1200 non-null   object
 10  accused_gender               1200 non-null   object
 11  prior_cases                  1200 non-null   object
 12  bail_outcome                 1200 non-null   object
 13  bail_outcome_label_detailed  1200

In [5]:
for col in df_raw.select_dtypes(include=['object']).columns:
    if (len(df_raw[col].unique()) <= 30):
        print(f"{col}: {df_raw[col].unique()}")
df_raw['court'].size

bail_type: ['Regular' 'Anticipatory' 'Interim' 'Unknown' 'Others' 'Not applicable']
accused_gender: ['Male' 'Female' 'Unknown' 'Multiple']
prior_cases: ['Unknown' 'Yes' 'No']
bail_outcome: ['Rejected' 'Granted']
crime_type: ['Narcotics' 'Sexual Offense' 'Fraud or Cheating' 'Others' 'Murder'
 'Dowry Harassment' 'Theft or Robbery' 'Kidnapping' 'Attempt to Murder'
 'Domestic Violence' 'Cyber Crime' 'Extortion']
region: ['Assam' 'Tamil Nadu' 'Kerala' 'West Bengal' 'Jammu & Kashmir' 'Punjab'
 'Karnataka' 'Maharashtra' 'Himachal Pradesh' 'Uttar Pradesh' 'Gujarat'
 'Madhya Pradesh' 'Haryana' 'Delhi' 'Uttarakhand' 'Bihar' 'Manipur'
 'Chhattisgarh' 'Odisha' 'Tripura' 'Rajasthan' 'Telangana' 'Jharkhand'
 'Andhra Pradesh' 'Ladakh' 'Chandigarh' 'Puducherry' 'Punjab and Haryana']
special_laws: [nan "['MCOCA Section 3 & 4', 'Arms Act Sections 25 & 27']"
 "['MCOCA Section 3(1)(i)', '3(2)', '3(4)', 'Arms Act Sections 3', '25', '27']"
 "['NDPS Act Sections 21', '29', 'Arms Act Sections 25', '27', 'MCOC

1200

In [6]:
print(df_raw["crime_type"].value_counts())

crime_type
Narcotics            134
Theft or Robbery     128
Dowry Harassment     123
Sexual Offense       121
Fraud or Cheating    121
Cyber Crime          116
Extortion            106
Kidnapping            95
Murder                89
Others                75
Domestic Violence     72
Attempt to Murder     20
Name: count, dtype: int64


In [7]:
df_raw['court_level'] = df_raw['court'].apply(lambda x: 1 if 'SUPREME COURT' in x.upper() else 2 if 'HIGH COURT' in x.upper() else 3)

In [8]:
df = df_raw.drop(df_raw.columns[[0, 1, 2, 3, 4, 9, 13, 16, 17, 18, 24, 23, 21]], axis=1).copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ipc_sections            1200 non-null   object
 1   bail_type               1200 non-null   object
 2   bail_cancellation_case  1200 non-null   bool  
 3   landmark_case           1200 non-null   bool  
 4   accused_gender          1200 non-null   object
 5   prior_cases             1200 non-null   object
 6   bail_outcome            1200 non-null   object
 7   crime_type              1200 non-null   object
 8   facts                   1200 non-null   object
 9   bias_flag               1200 non-null   bool  
 10  parity_argument_used    1200 non-null   bool  
 11  region                  1200 non-null   object
 12  court_level             1200 non-null   int64 
dtypes: bool(4), int64(1), object(8)
memory usage: 89.2+ KB


In [9]:
df["bail_cancellation_case"].value_counts()

bail_cancellation_case
False    1084
True      116
Name: count, dtype: int64

In [10]:
df.head()

,ipc_sections,bail_type,bail_cancellation_case,landmark_case,accused_gender,prior_cases,bail_outcome,crime_type,facts,bias_flag,parity_argument_used,region,court_level
0,"['120B', '121', '121A']",Regular,True,True,Male,Unknown,Rejected,Narcotics,Jibangshu Paul was apprehended carrying Rs. 32...,False,False,Assam,2
1,"['376', '354', '343', '109', '220', '348', '33...",Regular,True,False,Male,Unknown,Rejected,Sexual Offense,The case involves custodial rape of a woman by...,False,False,Tamil Nadu,2
2,"['465', '468', '471', '474', '420', '511', '34']",Anticipatory,False,True,Male,Unknown,Rejected,Fraud or Cheating,"Hyderali, a government contractor, was accused...",False,True,Kerala,2
3,"['326', '307', '120B', '201']",Regular,True,False,Male,Unknown,Granted,Others,"The petitioner, a government employee, alleged...",False,False,West Bengal,2
4,"['302', '34']",Regular,False,False,Female,Unknown,Rejected,Murder,Shankri Devi and co-accused were charged with ...,False,False,Jammu & Kashmir,2


In [11]:
df.court_level.value_counts()

court_level
2    1148
1      47
3       5
Name: count, dtype: int64

In [12]:
df.to_csv("./dataset/indian_bail_judgements_processed.csv", index=False)

In [13]:
df['prior_cases'].value_counts()

prior_cases
Unknown    583
Yes        401
No         216
Name: count, dtype: int64

In [14]:
for col in df.columns:
    if df[col].unique().shape[0] <= 10:
        print(f"{col}: {df[col].unique()}")

bail_type: ['Regular' 'Anticipatory' 'Interim' 'Unknown' 'Others' 'Not applicable']
bail_cancellation_case: [ True False]
landmark_case: [ True False]
accused_gender: ['Male' 'Female' 'Unknown' 'Multiple']
prior_cases: ['Unknown' 'Yes' 'No']
bail_outcome: ['Rejected' 'Granted']
bias_flag: [False  True]
parity_argument_used: [False  True]
court_level: [2 1 3]


In [15]:
import ast
df['ipc_sections'] = df['ipc_sections'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [16]:
ipc_sections = sorted(df['ipc_sections'].explode().dropna().astype(str).unique())

In [17]:
import json
with open("artefacts/ipc_sections.json", "w") as f:
    json.dump({'ipc_sections_list': ipc_sections}, f, indent=2)

In [18]:
df['accused_gender'].value_counts()

accused_gender
Male        1071
Female       123
Unknown        3
Multiple       3
Name: count, dtype: int64

In [19]:
input_df_1 = {
    "ipc_sections": ["323", "504", "34"],
    "bail_type": "Regular",
    "accused_gender": "Male",
    "prior_cases": "No",
    "crime_type": "Others",
    "region": "Delhi",
    "court_level": 3,
    "bail_cancellation_case": False,
    "landmark_case": False,
    "parity_argument_used": True,
    "bias_flag": False,
    "facts": "The accused is charged with minor non violent offences arising out of a neighborhood dispute. Investigation is complete and the accused has cooperated with authorities."
}

In [20]:
from collections import Counter
import joblib
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer

def lemmatize_words(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def normalize_ipc(x):
    if isinstance(x, list):
        return x
    if isinstance(x, (set, tuple)):
        return list(x)
    if pd.isna(x):
        return []
    return []


def preprocessing_pipeline(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the input DataFrame by handling missing values,
    encoding categorical variables, and vectorizing text data.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame with raw data.
    
    Returns:
    pd.DataFrame: Preprocessed DataFrame ready for modeling.
    """
    
    # Create a copy to avoid modifying original
    df = df.copy()
    
    # ========== 1. ONE-HOT ENCODING ==========
    #ohe_columns = joblib.load("artefacts/ohe_columns.pkl")
    
    # CRITICAL FIX: Remove duplicates from saved OHE columns
    #ohe_columns_unique = list(dict.fromkeys(ohe_columns))
    
    ohe_cols = ['bail_type', 'crime_type', 'region', 'accused_gender', 'prior_cases']
    df_ohe = pd.get_dummies(df[ohe_cols], drop_first=True, prefix=ohe_cols)
    
    # Reindex with deduplicated columns
    #df_ohe = df_ohe.reindex(columns=ohe_columns_unique, fill_value=0)
    
    # Drop original categorical columns
    df = df.drop(columns=ohe_cols)
    df = pd.concat([df.reset_index(drop=True),
                    df_ohe.reset_index(drop=True)], axis=1)
    
    # ========== 2. BOOLEAN ENCODING ==========
    bool_cols = ['bail_cancellation_case', 'landmark_case',
                 'parity_argument_used', 'bias_flag']
    
    for col in bool_cols:
        if col in df.columns:
            df[col] = df[col].replace({
                'True': 1, 'False': 0,
                'Yes': 1, 'No': 0,
                True: 1, False: 0
            }).fillna(0).astype(int)
    
    # ========== 3. IPC SECTIONS ENCODING ==========
    mlb = joblib.load('./artefacts/ipc_mlb.pkl')
    
    df['ipc_sections'] = df['ipc_sections'].apply(normalize_ipc)
    ipc_encoded = mlb.transform(df['ipc_sections'])
    
    # Create IPC DataFrame with unique column names
    ipc_columns = [c for c in mlb.classes_]
    ipc_df = pd.DataFrame(ipc_encoded, columns=ipc_columns, index=df.index)
    
    df = df.drop(columns=['ipc_sections'])
    df = pd.concat([df, ipc_df], axis=1)
    # ========== 4. TF-IDF VECTORIZATION ==========
    tfidf = joblib.load('./artefacts/tfidf_vectorizer.pkl')
    
    df['facts_clean'] = df['facts'].apply(clean_text)
    df['facts_clean'] = df['facts_clean'].apply(lemmatize_words)
    
    tfidf_matrix = tfidf.transform(df['facts_clean'])
    
    # CRITICAL FIX: Get feature names and handle potential duplicates properly
    tfidf_features = tfidf.get_feature_names_out()
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_features, index=df.index)
    # Drop original text columns
    df = df.drop(columns=['facts', 'facts_clean'], errors='ignore')
    df = pd.concat([df.reset_index(drop=True),
                    tfidf_df.reset_index(drop=True)], axis=1)
    
    df_final = df.copy()
    return df


def predict_bail_score(df: pd.DataFrame, mlp) -> float:
    """
    Predict bail scores using the trained model.
    
    Parameters:
    df (pd.DataFrame): Preprocessed DataFrame
    mlp: Trained model object
    
    Returns:
    numpy.ndarray: Predicted bail scores
    """
    return mlp.predict(df)

In [21]:
temp = preprocessing_pipeline(input_df_1)

TypeError: unhashable type: 'list'

In [ ]:
temp.head()

,court_level,bail_cancellation_case,landmark_case,parity_argument_used,bias_flag,ipc_sections,bail_outcome,facts,bail_type_Interim,bail_type_Not applicable,...,yadav wa,year,year custody,year later,year marriage,year month,year old,year time,year trial,youth
0,3,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
